In [ ]:
%pip install langchain langchain_openai openai --upgrade

In [2]:
import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

client = OpenAI()

speech_file_path = "speech.mp3"
response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input="Today is a wonderful day to build something people love!",
)

# Stream the response to a file
with open(speech_file_path, "wb") as file:
    for chunk in response.iter_bytes():
        file.write(chunk)

In [9]:
# Let's automate a BBC News Article and Generate The Audio for It:
from langchain_community.document_loaders import WebBaseLoader

url = 'https://www.bbc.com/sport/live/cvg1zx59kgjt'

loader = WebBaseLoader(url)

# Extract the text:
data = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [10]:
# Joining all of the documents and removing any redundant characters:
web_text = " ".join([doc.page_content for doc in data]).replace('\n', '')

In [11]:
len(web_text) # Checking the character length.

20650

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate

# Map
map_template = """The following is a set of documents for a BBC News Article:
{text}
Based on this list of docs, please extract the story and make it sound engaging:
"""
map_prompt = ChatPromptTemplate.from_template(map_template)

# If over 4090 characters, then summarize:
if len(web_text) > 4090:
    chain = load_summarize_chain(ChatOpenAI(), 
                                 chain_type="map_reduce", 
                                 map_prompt=map_prompt, 
                                 combine_prompt=map_prompt)
    
    result = chain.invoke({"input_documents": [Document(page_content=web_text)]})
    web_text = result['output_text']

# TODO - You might also want to clean up the text anyway to ensure an effective story is generated through the 
# text to speech

In [20]:
print(web_text)

Max Verstappen stole the show at the F1 Japanese Grand Prix, delivering a breathtaking performance to claim victory for Red Bull. In a thrilling race filled with fierce competition, the Dutch driver held off strong challenges from McLaren's Lando Norris and Oscar Piastri to secure his first win of the season. With a masterful display of composure and skill, Verstappen's dominance on the track was undeniable.

Despite a close call when Norris was forced onto the grass at the pit-lane exit, Verstappen's stellar pole position lap on Saturday proved to be a game-changer. McLaren's Piastri also put up a valiant effort, finishing in third place. Verstappen's triumph in Japan not only brings him within striking distance of Norris in the drivers' championship but also marks his fourth consecutive win at Suzuka.

As the F1 circuit gears up for the next race in Bahrain, the stage is set for an exciting battle between Verstappen and Norris as they continue their quest for the championship title. 

In [22]:
speech_file_path = "speech.mp3"
response = client.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=web_text,
)

# Stream the response to a file
with open(speech_file_path, "wb") as file:
    for chunk in response.iter_bytes():
        file.write(chunk)